In [6]:
import pandas as pd
import sys
from imp import reload
sys.path.insert(0, "/media/diskD/EPFL/Fall 2016/ADA/Project/GMR_ADA_Project/EmotionAnalysis")
reload(sys)
#from DataSchemaExtractionParsing import *
#from DataPreProcessing import *
#from SentSemanticModule import *
#from SentTweetModule import *
#from SentSyntacticModule import *

In [25]:
word_set = list(set(lexicon['Word']))
word_emotional_vectors = []
for word in word_set:
    word_emotional_vectors.append((word,list(lexicon[lexicon['Word']==word]['Score'])))
word_emotional_vectors_dict = dict(word_emotional_vectors)

[('fawn', ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0']),
 ('unscientific', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('cussed', ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('middleman', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('foul', ['1', '0', '1', '1', '0', '1', '0', '0', '0', '0']),
 ('narcotic', ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0']),
 ('prefix', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('aegis', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('hanging', ['1', '0', '1', '1', '0', '1', '0', '1', '0', '0']),
 ('conjuring', ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0']),
 ('woody', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('centimeter', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('vascular', ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']),
 ('aggression', ['1', '0', '0', '1', '0', '1', '0', '0', '0', '0']),
 ('conjure', ['0', '1', '0', '0', '0', '0', '0', '0', '1', '0']),
 ('el

# Data Extraction & Parsing

In [ ]:
sample_tweets = extract_tweets('Data/sample.tsv.xlsx')
# Keeping Meaningful columns
sample_tweets = sample_tweets[['createdAt', 'text', 'longitude', 'latitude', 'placeLongitude', 'placeLatitude', 'userLocation']]
sample_tweets.head()

In [ ]:
sample_tweets.shape

In [ ]:
set(sample_tweets['userLocation'])

In [ ]:
sample_tweets.columns

In [ ]:
#sample_tweets_cleaned = cleanTweets(sample_tweets)
sample_tweets_cleaned.head()

In [ ]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
sample_tweets['text']

In [ ]:
#sample_tweets = sample_tweets.dropna()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2891]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2890]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[2891]]).reindex() 
#sample_tweets = sample_tweets.drop(sample_tweets.index[[3916]]).reindex() 
#sample_tweets = sample_tweets.drop(sample_tweets.index[[4311]]).reindex()
#sample_tweets = sample_tweets.drop(sample_tweets.index[[7599]]).reindex()
sample_tweets.isnull().any()

In [ ]:
sample_tweets.iloc[864]

# Pre-Processing and Cleaning Tweets:

### Strategy:
1. Parsing and Character Encoding
2. Language Detection (Translation)
3. Remove Tweets for which geolocation (longitude  is not present or where geolocation does not correspond to Switzerland
4. Handling Entities/ Special categories: 
    4.1. Replacing @ instances with <username>
    4.2. Replacing urls with <url>
    4.3. Replacing Emoticons with their word meaning
    4.4. Replacing numbers/phone/fax with <number>
    4.5. Detecting place / city / country / any geolocation cues in any part of the tweet (#)
    4.6. Detecting time cues
5. Tokenization and replacing contractions
6. Part of Speech Tagging to recognize Affective words (Noun, Verbs, Adjectives, Adverb) 
7. Some NRE to replace basic entities like Proper Nouns with tag <proper_noun>
8. Stopwords and punctuation removal
9. (possibly spell-checking as well)
10. Lowering multiple occurences of a character in a word (words like soooooo => so)
11. Lemmatization and term normalization to get less variable versions of the same word. (possibly use thesaurus also) 
12. Remove less frequent words => word count + define a specific threshold

### 2. Language Detection & Translation

In [ ]:
from yandex_translate import YandexTranslate
translate = YandexTranslate('trnsl.1.1.20161128T210956Z.66d67a07332e46b7.66bee9bcc58c60428596f3e559b4a75a1944f839')

translated_list = []
for i in range(0,len(sample_tweets)):
    print i
    translated_list.append(translate.translate(sample_tweets.iloc[i]['text'], 'en')['text'])
sample_tweets['EnglishText'] = translated_list
sample_tweets.head()

### 3. Remove Tweets for which geolocation is not present or where geolocation does not correspond to Switzerland

### 4. Handling Special Entities / Categories:

#### 4.2. Replacing URLs in Tweets

### Tokenization of Tweets into words

In [ ]:
from nltk.tokenize import word_tokenize
 
word_tokenize(sample_tweets['text'])